In [ ]:
### From this source : https://towardsdatascience.com/how-to-extract-keywords-from-pdfs-and-arrange-in-order-of-their-weights-using-python-841556083341

In [ ]:
### Importing the appropriate packages

!pip install PyPDF2
!pip install textract
!pip install regex

In [10]:
### importing the appropriate libraries

import pandas as pd
import numpy as np
import PyPDF2
import textract
import re

In [7]:
filename ='/Users/etiennecampredon/Projects/pharmacy_industry/annual_reports/sanofi_SR_21.pdf' 

pdfFileObj = open(filename,'rb')               #open allows you to read the file
pdfReader = PyPDF2.PdfReader(pdfFileObj)   #The pdfReader variable is a readable object that will be parsed
num_pages = len(pdfReader.pages)                 #discerning the number of pages will allow us to parse through all the pages


count = 0
text = ""
                                                            
while count < num_pages:                       #The while loop will read each page
    pageObj = pdfReader.pages[count]
    count +=1
    text += pageObj.extract_text()
    
#Below if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.

if text != "":
    text = text
    
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text

else:
    text = textract.process('http://bit.ly/epo_keyword_extraction_document', method='tesseract', language='eng')

    # Now we have a text variable which contains all the text derived from our PDF file.

In [8]:
text = text.encode('ascii','ignore').lower() #Lowercasing each word

In [13]:
keywords = re.findall(r'[a-zA-Z]\w+',text.decode('utf-8'))
len(keywords)                               #Total keywords in document

50867

In [14]:
df = pd.DataFrame(list(set(keywords)),columns=['keywords'])  #Dataframe with unique keywords to avoid repetition in rows

In [15]:
def weightage(word,text,number_of_documents=1):
    word_list = re.findall(word,text)
    number_of_times_word_appeared =len(word_list)
    tf = number_of_times_word_appeared/float(len(text))
    idf = np.log((number_of_documents)/float(number_of_times_word_appeared))
    tf_idf = tf*idf
    return number_of_times_word_appeared,tf,idf ,tf_idf 

In [17]:
df['number_of_times_word_appeared'] = df['keywords'].apply(lambda x: weightage(x,text.decode('utf-8'))[0])
df['tf'] = df['keywords'].apply(lambda x: weightage(x,text.decode('utf-8'))[1])
df['idf'] = df['keywords'].apply(lambda x: weightage(x,text.decode('utf-8'))[2])
df['tf_idf'] = df['keywords'].apply(lambda x: weightage(x,text.decode('utf-8'))[3])

In [19]:
df = df.sort_values('tf_idf',ascending=True)
df.to_csv('Keywords.csv')
df.head(100)

,keywords,number_of_times_word_appeared,tf,idf,tf_idf
2462,in,6159,0.016597,-8.725670,-0.144822
3945,an,5534,0.014913,-8.618666,-0.128530
3843,on,4543,0.012242,-8.421343,-0.103098
3008,at,4024,0.010844,-8.300032,-0.090004
2942,en,3969,0.010696,-8.286269,-0.088627
...,...,...,...,...,...
322,was,177,0.000477,-5.176150,-0.002469
1670,from,177,0.000477,-5.176150,-0.002469
790,ey,174,0.000469,-5.159055,-0.002419
3827,eu,174,0.000469,-5.159055,-0.002419


### Second Method - Using Gensim library

In [26]:
!pip install gensim == 3.8.3

zsh:1: = not found


In [25]:
!pip install gensim == 3.8.3
from gensim.summarization import keywords
import warnings
warnings.filterwarnings("ignore")

zsh:1: = not found


ModuleNotFoundError: No module named 'gensim.summarization'

In [ ]:
values = keywords(text=text,split='\n',scores=True)

In [ ]:
data = pd.DataFrame(values,columns=['keyword','score'])
data = data.sort_values('score',ascending=False)
data.head(10)